# Chapter 3 Exercise 1: Meta-Learners - 因果效应估计的瑞士军刀

## 学习目标

1. 理解 S-Learner 和 T-Learner 的原理
2. 实现基础的 Meta-Learner 算法
3. 理解不同方法的优缺点
4. 掌握 CATE (条件平均处理效应) 估计和评估

---

## 什么是 Meta-Learners?

想象你是一位医生，想知道某种新药对不同病人的效果。问题是：每个病人只能选择吃药或不吃药中的一种，你永远无法同时观察到两种结果。

**Meta-Learners** 就是用 "元学习" 的思想来解决这个问题——它们把因果效应估计转化为监督学习问题，让我们可以用任何机器学习模型（如 Random Forest、XGBoost）来估计处理效应！

这就像是给你一套万能工具，可以把各种 ML 模型变成因果推断工具。

## 生活化类比：餐厅会员卡的效果

假设你经营一家餐厅，推出了会员卡（处理 T）：
- **处理组 (T=1)**：拿到会员卡的顾客
- **控制组 (T=0)**：没有会员卡的顾客
- **结果 Y**：月消费金额

你想知道：**会员卡到底让顾客多花了多少钱？** 而且更进一步：**对哪些顾客效果最好？**

| 顾客类型 | 原本消费 | 有会员卡后消费 | 处理效应 |
|---------|---------|--------------|--------|
| 学生党 | 200 | 350 | **+150** (效果好!) |
| 上班族 | 500 | 520 | +20 (效果一般) |
| 土豪 | 2000 | 2010 | +10 (几乎没效果) |

这种 **因人而异** 的效应就叫做 **CATE (Conditional Average Treatment Effect)**——条件平均处理效应。

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from typing import Tuple

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

np.random.seed(42)
print("环境准备完成！")

---

## Part 1: S-Learner (Single Model Learner)

### 核心思想

S-Learner 是最简单的 Meta-Learner：

**把处理 T 当作普通特征，训练一个模型同时预测所有人的结果**

$$\hat{\mu}(x, t) = \hat{E}[Y | X=x, T=t]$$

然后，CATE 就是预测差值：

$$\hat{\tau}(x) = \hat{\mu}(x, 1) - \hat{\mu}(x, 0)$$

### 类比：一个班级，一位老师

想象一个班级里有男生和女生，老师要预测他们的考试成绩：

- **S-Learner 方法**：老师把 "性别" 作为一个特征，用同一套模型预测所有学生的成绩
- 然后比较：如果这个学生是男生会考多少分？如果是女生会考多少分？差值就是 "性别效应"

### S-Learner 的流程图

```
训练阶段:
┌─────────────────┐      ┌─────────────────┐
│  特征 X + 处理 T │ ───► │  单一模型 f(X,T) │ ───► 预测 Y
└─────────────────┘      └─────────────────┘

预测阶段:
                         ┌─ f(X, T=1) ─┐
┌──────┐                 │             │    减法
│ 特征 X│ ───────────────┤             ├─────────► CATE
└──────┘                 │             │
                         └─ f(X, T=0) ─┘
```

In [ ]:
# 练习 1.1: 实现 S-Learner

class SimpleSLearner:
    """
    S-Learner (Single Model Learner)
    
    核心思想:
    - 将处理 T 作为一个特征
    - 训练单一模型: Y = f(X, T)
    - CATE 估计: tau(x) = f(x, 1) - f(x, 0)
    """
    
    def __init__(self):
        # TODO: 初始化基础模型
        # 可以使用 LinearRegression 或 RandomForestRegressor
        self.model = None  # 你的代码
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """
        训练 S-Learner
        
        TODO:
        1. 将 T 和 X 合并为特征矩阵 (T 作为最后一列)
        2. 训练模型预测 Y
        
        提示: 使用 np.column_stack([X, T.reshape(-1, 1)]) 合并
        """
        # 你的代码
        pass
    
    def predict_cate(self, X: np.ndarray) -> np.ndarray:
        """
        预测 CATE (个体处理效应)
        
        TODO:
        1. 构造 T=1 的特征: 把 1 添加到 X 后面
        2. 构造 T=0 的特征: 把 0 添加到 X 后面
        3. 分别预测 Y(1) 和 Y(0)
        4. 返回差值: Y(1) - Y(0)
        """
        # 你的代码
        pass

---

## Part 2: T-Learner (Two Model Learner)

### 核心思想

T-Learner 认为：处理组和控制组可能遵循完全不同的规律，所以应该分开建模！

$$\hat{\mu}_0(x) = \hat{E}[Y | X=x, T=0] \quad \text{(控制组模型)}$$
$$\hat{\mu}_1(x) = \hat{E}[Y | X=x, T=1] \quad \text{(处理组模型)}$$

CATE 估计：
$$\hat{\tau}(x) = \hat{\mu}_1(x) - \hat{\mu}_0(x)$$

### 类比：两个班级，两位老师

- **T-Learner 方法**：男生和女生分成两个班，由不同的老师教
- 每位老师只了解自己班级的学生
- 比较两个老师的预测差值来估计 "性别效应"

### T-Learner 的流程图

```
训练阶段:
┌────────────────┐      ┌─────────────────┐
│ 控制组 (T=0)   │ ───► │  模型 0: μ₀(X)  │
└────────────────┘      └─────────────────┘

┌────────────────┐      ┌─────────────────┐
│ 处理组 (T=1)   │ ───► │  模型 1: μ₁(X)  │
└────────────────┘      └─────────────────┘

预测阶段:
┌──────┐      ┌─────────┐     
│特征 X│ ───► │ 模型 0  │ ───► Y(0)  ┐
└──────┘      └─────────┘            │ 减法
   │         ┌─────────┐            ├────► CATE
   └───────► │ 模型 1  │ ───► Y(1)  ┘
             └─────────┘
```

In [ ]:
# 练习 1.2: 实现 T-Learner

class SimpleTLearner:
    """
    T-Learner (Two Model Learner)
    
    核心思想:
    - 分别为处理组和控制组训练两个模型
    - mu_0(x) = E[Y|X=x, T=0]  (控制组模型)
    - mu_1(x) = E[Y|X=x, T=1]  (处理组模型)
    - CATE 估计: tau(x) = mu_1(x) - mu_0(x)
    """
    
    def __init__(self):
        # TODO: 初始化两个模型
        self.model_0 = None  # 控制组模型
        self.model_1 = None  # 处理组模型
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """
        训练 T-Learner
        
        TODO:
        1. 将数据按 T 分成两组 (mask_0 = T == 0, mask_1 = T == 1)
        2. 用控制组数据训练 model_0
        3. 用处理组数据训练 model_1
        """
        # 你的代码
        pass
    
    def predict_cate(self, X: np.ndarray) -> np.ndarray:
        """
        预测 CATE
        
        TODO:
        1. 使用 model_1 预测 Y(1)
        2. 使用 model_0 预测 Y(0)
        3. 返回差值 Y(1) - Y(0)
        """
        # 你的代码
        pass

---

## Part 3: 生成 Uplift 数据

### 数据生成过程 (DGP) 的重要性

在因果推断中，我们经常需要用模拟数据来验证方法是否有效。因为在模拟数据中，我们 **知道真实的因果效应**！

### 我们的 DGP

假设我们在研究「课外补习」对「考试成绩」的效果：

- **X1**: 学生的学习能力 (标准正态分布)
- **X2**: 学生的学习时间 (标准正态分布)
- **T**: 是否参加补习 (随机分配)
- **Y(0)**: 不补习的成绩 = $5 + 2 \cdot X_1 + X_2 + \epsilon$
- **Y(1)**: 补习后的成绩 = $Y(0) + \tau(X)$

其中，**异质性效应**：
$$\tau(X) = 2 + X_1 - 0.5 \cdot X_2$$

这意味着：
- 学习能力强的学生 (X1 高) → 补习效果更好
- 学习时间长的学生 (X2 高) → 补习效果稍差 (因为已经很努力了)
- 平均处理效应 ATE ≈ 2

In [ ]:
# 练习 1.3: 生成 Uplift 数据

def generate_simple_uplift_data(
    n: int = 1000,
    heterogeneous: bool = True,
    seed: int = 42
) -> Tuple[pd.DataFrame, np.ndarray]:
    """
    生成简单的 Uplift 数据
    
    数据生成过程 (DGP):
    - X1, X2 ~ N(0, 1)
    - T ~ Bernoulli(0.5)  (随机分配)
    - Y(0) = 5 + 2*X1 + X2 + noise
    - Y(1) = Y(0) + tau(X)
    
    其中:
    - 如果 heterogeneous=True: tau(X) = 2 + X1 - 0.5*X2  (异质性效应)
    - 如果 heterogeneous=False: tau(X) = 2  (常数效应)
    
    TODO: 完成数据生成代码
    
    Returns:
        (DataFrame, true_cate)
        DataFrame columns: X1, X2, T, Y
        true_cate: 真实的个体处理效应
    """
    np.random.seed(seed)
    
    # TODO: 生成特征 X1, X2 (标准正态分布)
    X1 = None  # 你的代码: np.random.randn(n)
    X2 = None  # 你的代码: np.random.randn(n)
    
    # TODO: 随机分配处理 T (伯努利分布, p=0.5)
    T = None  # 你的代码: np.random.binomial(1, 0.5, n)
    
    # TODO: 计算真实 CATE
    if heterogeneous:
        true_cate = None  # 异质性: 2 + X1 - 0.5*X2
    else:
        true_cate = None  # 常数: np.ones(n) * 2
    
    # TODO: 生成潜在结果
    noise = np.random.randn(n) * 0.5
    # Y(0) = 5 + 2*X1 + X2 + noise
    Y0 = None  # 你的代码
    # Y(1) = Y(0) + true_cate
    Y1 = None  # 你的代码
    # 观测到的 Y = T * Y(1) + (1-T) * Y(0)
    Y = None  # 你的代码
    
    df = pd.DataFrame({
        'X1': X1,
        'X2': X2,
        'T': T,
        'Y': Y
    })
    
    return df, true_cate

# 测试数据生成
df, true_cate = generate_simple_uplift_data(n=100)
if df is not None and true_cate is not None:
    print(f"样本量: {len(df)}")
    print(f"真实平均 CATE (ATE): {true_cate.mean():.4f}")
    print(f"CATE 范围: [{true_cate.min():.2f}, {true_cate.max():.2f}]")
    print("\n数据预览:")
    display(df.head())
else:
    print("[未完成] 请完成 generate_simple_uplift_data 函数")

---

## Part 4: 评估 CATE 估计质量

### 关键指标

| 指标 | 公式 | 含义 |
|------|------|------|
| **MSE** | $\frac{1}{n}\sum(\tau_{true} - \tau_{pred})^2$ | 均方误差，越小越好 |
| **MAE** | $\frac{1}{n}\sum|\tau_{true} - \tau_{pred}|$ | 平均绝对误差 |
| **Correlation** | $corr(\tau_{true}, \tau_{pred})$ | 相关性，越接近1越好 |
| **PEHE** | $\sqrt{MSE}$ | Precision in Estimation of HTE，最常用指标 |

### PEHE 的重要性

**PEHE (Precision in Estimation of Heterogeneous Effect)** 是评估 CATE 估计的黄金标准：

$$PEHE = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(\tau_i^{true} - \tau_i^{pred})^2}$$

它告诉我们：**模型预测的个体效应与真实效应有多接近？**

In [ ]:
# 练习 1.4: 评估 CATE 估计

def evaluate_cate_estimation(
    true_cate: np.ndarray,
    predicted_cate: np.ndarray
) -> dict:
    """
    评估 CATE 估计的质量
    
    TODO: 计算以下指标:
    1. MSE: 均方误差 = mean((true - pred)^2)
    2. MAE: 平均绝对误差 = mean(|true - pred|)
    3. Correlation: 与真实 CATE 的相关系数
    4. PEHE: sqrt(MSE)
    
    Returns:
        字典包含所有指标
    """
    metrics = {
        'MSE': None,        # 你的代码: np.mean((true_cate - predicted_cate)**2)
        'MAE': None,        # 你的代码: np.mean(np.abs(true_cate - predicted_cate))
        'Correlation': None,# 你的代码: np.corrcoef(true_cate, predicted_cate)[0, 1]
        'PEHE': None        # 你的代码: np.sqrt(MSE)
    }
    
    return metrics

# 测试评估函数
if true_cate is not None:
    # 用一个随机预测测试
    random_pred = np.random.randn(len(true_cate)) * 2 + 2
    metrics = evaluate_cate_estimation(true_cate, random_pred)
    if metrics['MSE'] is not None:
        print("随机预测的评估结果:")
        for key, value in metrics.items():
            print(f"  {key}: {value:.4f}")
    else:
        print("[未完成] 请完成 evaluate_cate_estimation 函数")

---

## Part 5: S-Learner vs T-Learner 实战对比

### 两种方法的对比

| 特性 | S-Learner | T-Learner |
|------|-----------|------------|
| 模型数量 | 1个 | 2个 |
| 样本利用 | 使用全部数据 | 分组使用 |
| 处理效应 | 从模型系数中学习 | 直接比较预测值 |
| 正则化偏差 | 可能低估效应 | 较小 |
| 方差 | 较低 | 较高 (特别是样本不平衡时) |

### 什么时候用哪个？

- **S-Learner 更好**: 处理效应较小、样本量有限、处理组和控制组相似
- **T-Learner 更好**: 处理效应显著、两组规律不同、异质性强

In [ ]:
# 练习 1.5: 比较 S-Learner 和 T-Learner

def compare_s_and_t_learner(
    n_samples: int = 2000,
    heterogeneous: bool = True
) -> pd.DataFrame:
    """
    比较 S-Learner 和 T-Learner 的性能
    
    TODO:
    1. 生成数据
    2. 训练 S-Learner 并预测 CATE
    3. 训练 T-Learner 并预测 CATE
    4. 评估两者的 CATE 估计
    5. 返回对比结果
    
    Returns:
        DataFrame with comparison results
    """
    # TODO: 生成数据
    df, true_cate = generate_simple_uplift_data(n_samples, heterogeneous)
    
    X = df[['X1', 'X2']].values
    T = df['T'].values
    Y = df['Y'].values
    
    results = []
    
    # TODO: 训练 S-Learner
    s_learner = SimpleSLearner()
    # s_learner.fit(X, T, Y)
    # s_pred = s_learner.predict_cate(X)
    s_pred = None  # 你的代码
    
    # TODO: 训练 T-Learner
    t_learner = SimpleTLearner()
    # t_learner.fit(X, T, Y)
    # t_pred = t_learner.predict_cate(X)
    t_pred = None  # 你的代码
    
    # TODO: 评估并收集结果
    if s_pred is not None:
        s_metrics = evaluate_cate_estimation(true_cate, s_pred)
        s_metrics['Method'] = 'S-Learner'
        results.append(s_metrics)
    
    if t_pred is not None:
        t_metrics = evaluate_cate_estimation(true_cate, t_pred)
        t_metrics['Method'] = 'T-Learner'
        results.append(t_metrics)
    
    return pd.DataFrame(results) if results else None, s_pred, t_pred, true_cate

# 运行对比实验
print("=" * 50)
print("场景 1: 异质性效应")
print("=" * 50)
result, s_pred, t_pred, true_cate = compare_s_and_t_learner(n_samples=1000, heterogeneous=True)
if result is not None:
    display(result)
else:
    print("[未完成] 请完成所有练习后重新运行")

print("\n" + "=" * 50)
print("场景 2: 常数效应")
print("=" * 50)
result2, _, _, _ = compare_s_and_t_learner(n_samples=1000, heterogeneous=False)
if result2 is not None:
    display(result2)

---

## Part 6: 可视化 CATE 估计结果

让我们可视化模型的预测效果！

In [ ]:
# 可视化 CATE 估计结果

if s_pred is not None and t_pred is not None:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 图1: 真实 vs S-Learner
    axes[0].scatter(true_cate, s_pred, alpha=0.5, s=10)
    axes[0].plot([true_cate.min(), true_cate.max()], 
                 [true_cate.min(), true_cate.max()], 'r--', lw=2)
    axes[0].set_xlabel('True CATE')
    axes[0].set_ylabel('Predicted CATE')
    axes[0].set_title('S-Learner: True vs Predicted')
    
    # 图2: 真实 vs T-Learner
    axes[1].scatter(true_cate, t_pred, alpha=0.5, s=10)
    axes[1].plot([true_cate.min(), true_cate.max()], 
                 [true_cate.min(), true_cate.max()], 'r--', lw=2)
    axes[1].set_xlabel('True CATE')
    axes[1].set_ylabel('Predicted CATE')
    axes[1].set_title('T-Learner: True vs Predicted')
    
    # 图3: CATE 分布对比
    axes[2].hist(true_cate, bins=30, alpha=0.5, label='True CATE', density=True)
    axes[2].hist(s_pred, bins=30, alpha=0.5, label='S-Learner', density=True)
    axes[2].hist(t_pred, bins=30, alpha=0.5, label='T-Learner', density=True)
    axes[2].set_xlabel('CATE')
    axes[2].set_ylabel('Density')
    axes[2].set_title('CATE Distribution')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("请先完成上面的练习，生成预测结果")

---

## 思考题

请在下面的 markdown 单元格中写下你的答案：

### 1. S-Learner 的优点和缺点是什么？

**你的答案:**

优点:
- 
- 

缺点:
- 
- 

### 2. T-Learner 的优点和缺点是什么？

**你的答案:**

优点:
- 
- 

缺点:
- 
- 

### 3. 在什么情况下 S-Learner 可能表现更好？在什么情况下 T-Learner 可能表现更好？

**你的答案:**

S-Learner 更好:
- 

T-Learner 更好:
- 

### 4. X-Learner 相比 S-Learner 和 T-Learner 有什么创新之处？

**你的答案:**
- 
- 

### 5. 如果处理组和控制组的样本量严重不平衡 (比如 90% vs 10%)，应该选择哪种 Meta-Learner？为什么？

**你的答案:**



---

## 总结

### 今天学到了什么？

| 概念 | 要点 |
|------|------|
| **Meta-Learners** | 把因果推断问题转化为监督学习问题的通用方法 |
| **S-Learner** | 单模型，把 T 当特征，简单但可能低估效应 |
| **T-Learner** | 双模型，分别建模，灵活但可能方差大 |
| **CATE** | 条件平均处理效应，反映效应的异质性 |
| **PEHE** | 评估 CATE 估计的黄金标准 |

### 下一步

在下一个练习中，我们将学习 **Uplift Tree** —— 一种专门为因果推断设计的决策树！

---

*恭喜你完成了 Meta-Learners 的学习！* 🎉